In [ ]:
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import os
from tensorflow.keras.metrics import Precision, Recall, F1Score

# Directory to save models
model_dir = "pretrained_models"
os.makedirs(model_dir, exist_ok=True)

# Load SSD MobileNet model
ssd_model_path = os.path.join(model_dir, "ssd_mobilenet_v2")
if not os.path.exists(ssd_model_path):
    object_detection = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")
    tf.saved_model.save(object_detection, ssd_model_path)
else:
    object_detection = hub.load(ssd_model_path)
print("SSD MobileNet model loaded.")

# Load MoveNet model
movenet_model_path = os.path.join(model_dir, "movenet_thunder")
if not os.path.exists(movenet_model_path):
    posture_detection = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    tf.saved_model.save(posture_detection, movenet_model_path)
else:
    posture_detection = hub.load(movenet_model_path)
print("MoveNet model loaded.")

# Friendly QR data
FRIENDLY_QR_CODES = {"friend1", "friend2", "team123"}

# Camouflage detection model
camouflage_model = tf.keras.models.load_model(
    '"pretrained_models/Camoflauge/camoflague_detection.keras"',
    custom_objects={'f1_score': F1Score(), 'Precision': Precision(), 'Recall': Recall()}
)

# Hostility scoring parameters
WEAPON_WEIGHT = 0.5
POSTURE_WEIGHT = 0.3
GEAR_WEIGHT = 0.2
CAMOUFLAGE_WEIGHT = 0.3
HOSTILITY_THRESHOLD = 0.7

# Detection functions
def detect_objects(image, model):
    image_uint8 = tf.image.convert_image_dtype(image, tf.uint8)
    input_tensor = image_uint8[tf.newaxis, ...]
    detections = model(input_tensor)
    return detections

def detect_posture(image, model):
    infer = model.signatures['serving_default']
    input_image = tf.image.resize(image, [256, 256])
    input_image = tf.cast(input_image, dtype=tf.int32)
    input_image = input_image[tf.newaxis, ...]
    outputs = infer(input_image)
    keypoints = outputs['output_0'][0][0]
    return keypoints

def detect_camouflage(image):
    image_resized = cv2.resize(image, (128, 128))
    image_normalized = image_resized / 255.0
    mask = camouflage_model.predict(np.expand_dims(image_normalized, axis=0))
    return mask[0, :, :, 0]

def apply_camouflage_mask(frame, mask):
    mask_resized = cv2.resize(mask, (frame.shape[1], frame.shape[0]))
    mask_binary = (mask_resized > 0.5).astype(np.uint8)
    mask_colored = np.zeros_like(frame)
    mask_colored[:, :, 2] = mask_binary * 255  # Apply red color to the mask
    return cv2.addWeighted(frame, 1, mask_colored, 0.5, 0)

def classify_friend_or_foe(frame):
    qr_detector = cv2.QRCodeDetector()
    data, bbox, _ = qr_detector.detectAndDecode(frame)
    if data:
        return 'Friend' if data in FRIENDLY_QR_CODES else 'FOE', bbox
    return None, None

def calculate_hostility_score(detections, keypoints, camouflage_mask):
    hostility_score = 0
    weapon_detected = False
    for detection in detections['detection_classes'][0].numpy():
        if detection == 1:  # Weapon class in COCO
            weapon_detected = True
            hostility_score += WEAPON_WEIGHT
            break

    if keypoints is not None:
        left_wrist, right_wrist, nose = keypoints[9], keypoints[10], keypoints[0]
        if left_wrist[1] < nose[1] and right_wrist[1] < nose[1]:
            hostility_score += POSTURE_WEIGHT

    if camouflage_mask is not None and np.any(camouflage_mask > 0.5):
        hostility_score += CAMOUFLAGE_WEIGHT

    return hostility_score, weapon_detected



SSD MobileNet model loaded.
MoveNet model loaded.


Enter Video Path Below

In [ ]:
input_video = 'Examples/New Project - Made with Clipchamp.mp4'

In [ ]:

# Video processing

cap = cv2.VideoCapture(input_video)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_video_path = 'processed_video.mp4'
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width, frame_height = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    detections = detect_objects(rgb_frame, object_detection)
    keypoints = detect_posture(rgb_frame, posture_detection)
    camouflage_mask = detect_camouflage(frame)

    # Highlight camouflage in the frame
    frame = apply_camouflage_mask(frame, camouflage_mask)

    # Friend or Foe classification
    classification, bbox = classify_friend_or_foe(frame)
    hostility_score, weapon_detected = calculate_hostility_score(detections, keypoints, camouflage_mask)

    for i in range(int(detections['num_detections'][0].numpy())):
        detection_score = detections['detection_scores'][0][i].numpy()
        if detection_score > 0.5:
            class_id = int(detections['detection_classes'][0][i].numpy())
            ymin, xmin, ymax, xmax = detections['detection_boxes'][0][i].numpy()
            left, top, right, bottom = int(xmin * frame.shape[1]), int(ymin * frame.shape[0]), int(xmax * frame.shape[1]), int(ymax * frame.shape[0])
            color = (255, 0, 0) if classification == "Friend" else (0, 255, 0) if hostility_score < HOSTILITY_THRESHOLD else (0, 0, 255)

            cv2.rectangle(frame, (left, top), (right, bottom), color, 3)
            label = f"{classification} (Score: {hostility_score:.2f})" if classification else f"Hostility {hostility_score:.2f}"
            cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    out.write(frame)

cap.release()
out.release()

print('Output video saved at:', output_video_path)

I0000 00:00:1732627044.085457      64 service.cc:145] XLA service 0x7b8d08003060 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732627044.085511      64 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


I0000 00:00:1732627045.240200      64 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━